<a href="https://colab.research.google.com/github/harishkumar1218/harishkumar/blob/main/text_to_text_AI_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer



In [2]:
import pandas as pd

df = pd.read_csv('/content/sample_data/Conversation.csv') #downlaod csv from https://www.kaggle.com/datasets/kreeshrajani/3k-conversations-dataset-for-chatbot and its path


input_text = [x for x in df['question']]
output_text = [x for x in df['answer']]


In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenized_inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)
tokenized_outputs = tokenizer(output_text, return_tensors="tf", padding=True, truncation=True)


In [4]:

input_ids = tf.convert_to_tensor(tokenized_inputs["input_ids"])
output_ids = tf.convert_to_tensor(tokenized_outputs["input_ids"])

training_dataset = tf.data.Dataset.from_tensor_slices((input_ids, output_ids))


In [ ]:

model = TFGPT2LMHeadModel.from_pretrained('gpt2')

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

model.fit(training_dataset, epochs=5, batch_size=4) #increse epochs for more accuracy may be 10. it takes lot of time.


In [21]:
model.save_pretrained('sylvie_ai_tf')#saving my model as sylvie from loki web serise

In [27]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('/content/sylvie_ai_tf')#path to model

tokenizer.pad_token_id = tokenizer.eos_token_id
user_input = "how's it going?"
input_ids = tokenizer.encode(user_input, return_tensors='tf')


attention_mask = tf.ones(input_ids.shape, dtype=tf.int32)

generated_output = model.generate(input_ids, attention_mask=attention_mask, max_length=200, num_return_sequences=1)
print(generated_output)

decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)
print(decoded_output)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/sylvie_ai_tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tf.Tensor([[ 4919   338   340  1016    30    13    13    13    13 50256]], shape=(1, 10), dtype=int32)
how's it going?....
